# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
!pip install hvplot geoviews cartopy pyproj
!pip install --upgrade panel param
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy

import matplotlib.pyplot as plt
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_df = pd.read_csv("cities.csv")

# Display sample data
city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,4.55,93.0,100.0,14.98,TF,1.726707e+09
1,1,st anne,49.7150,-2.2042,17.09,94.0,75.0,8.23,GG,1.726707e+09
2,2,suva,-18.1416,178.4415,24.71,61.0,75.0,3.60,FJ,1.726707e+09
3,3,adamstown,-25.0660,-130.1015,21.35,76.0,41.0,2.00,PN,1.726707e+09
4,4,tolanaro,-25.0319,46.9987,22.04,88.0,0.0,8.23,MG,1.726707e+09


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

ImportError: cannot import name 'param_value_if_widget' from 'panel.depends' (/opt/anaconda3/envs/dev/lib/python3.10/site-packages/panel/depends.py)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_df = city_df.loc[(city_df["Max Temp"] < 30) & (city_df["Max Temp"] > 15) & (city_df["Wind Speed"] < 10) & (city_df["Cloudiness"] < 15)]

# Drop any rows with null values
narrowed_df = narrowed_df.dropna()

# Display sample data
narrowed_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,tolanaro,-25.0319,46.9987,22.04,88.0,0.0,8.23,MG,1.726707e+09
6,6,atafu village,-8.5421,-172.5159,27.57,69.0,4.0,0.93,TK,1.726707e+09
24,24,barah,13.7000,30.3667,28.42,61.0,1.0,3.25,SD,1.726707e+09
28,28,shimo-furano,43.3500,142.3833,20.57,59.0,1.0,4.29,JP,1.726707e+09
30,30,taltal,-25.4000,-70.4833,16.21,71.0,0.0,2.56,CL,1.726707e+09
...,...,...,...,...,...,...,...,...,...,...
587,587,pitangui,-19.6828,-44.8903,22.57,57.0,1.0,4.46,BR,1.726707e+09
589,589,mulayjah,27.2710,48.4242,28.67,27.0,0.0,4.20,SA,1.726707e+09
591,591,mahanje,-9.9333,35.3333,15.13,79.0,0.0,1.36,TZ,1.726707e+09
594,594,pasni,25.2631,63.4710,26.43,88.0,0.0,2.05,PK,1.726707e+09


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,tolanaro,MG,-25.0319,46.9987,88.0,
6,atafu village,TK,-8.5421,-172.5159,69.0,
24,barah,SD,13.7000,30.3667,61.0,
28,shimo-furano,JP,43.3500,142.3833,59.0,
30,taltal,CL,-25.4000,-70.4833,71.0,
...,...,...,...,...,...,...
587,pitangui,BR,-19.6828,-44.8903,57.0,
589,mulayjah,SA,27.2710,48.4242,27.0,
591,mahanje,TZ,-9.9333,35.3333,79.0,
594,pasni,PK,25.2631,63.4710,88.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tolanaro - nearest hotel: Hôtel Mahavokey
atafu village - nearest hotel: No hotel found
barah - nearest hotel: No hotel found
shimo-furano - nearest hotel: Natulux Hotel Furano
taltal - nearest hotel: Hostería Taltal
yaren - nearest hotel: Aiwo Hotel
remire-montjoly - nearest hotel: Complexe Belova
irymple - nearest hotel: Chaffey Motor Inn
barpeta - nearest hotel: Manas Hotel
yarmouth - nearest hotel: The Inn At Cape Cod
broken hill - nearest hotel: Ibis Styles
curuca - nearest hotel: No hotel found
beberibe - nearest hotel: Pousada Ibitu
beohari - nearest hotel: No hotel found
saint-philippe - nearest hotel: Le Baril
forster - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
bethel - nearest hotel: Hampton Inn Danbury
turaif - nearest hotel: المودة
al kharijah - nearest hotel: Kharga Hotel
saint-joseph - nearest hotel: PALM Hotel & Spa
tsiombe - nearest hotel: No hotel found
fale old settlement - nearest hotel: No hotel found
ribeira

,City,Country,Lat,Lng,Humidity,Hotel Name
4,tolanaro,MG,-25.0319,46.9987,88.0,Hôtel Mahavokey
6,atafu village,TK,-8.5421,-172.5159,69.0,No hotel found
24,barah,SD,13.7000,30.3667,61.0,No hotel found
28,shimo-furano,JP,43.3500,142.3833,59.0,Natulux Hotel Furano
30,taltal,CL,-25.4000,-70.4833,71.0,Hostería Taltal
...,...,...,...,...,...,...
587,pitangui,BR,-19.6828,-44.8903,57.0,Hotel Santa Felicidade
589,mulayjah,SA,27.2710,48.4242,27.0,No hotel found
591,mahanje,TZ,-9.9333,35.3333,79.0,No hotel found
594,pasni,PK,25.2631,63.4710,88.0,Juddi Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

ImportError: cannot import name 'param_value_if_widget' from 'panel.depends' (/opt/anaconda3/envs/dev/lib/python3.10/site-packages/panel/depends.py)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)